In [18]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

In [19]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [26]:
for source_water in ["municipal"]: #df.source_water_category.unique():
    for case_study in df[df.source_water_category == source_water].case_study.unique():
        h1 = df[df.case_study == case_study]
        scenarios = h1.scenario.unique()
        for scenario in scenarios:

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()

            m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = wt.case_study_trains.get_case_study(m=m)

#             for key in m.fs.pfd_dict.keys():
#                 if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
#                     getattr(m.fs, key).a.fix(4.2)
#                     getattr(m.fs, key).b.fix(0.35)

            if "1p5" in scenario:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)
            
            ############################################################
            ############################################################
            # set recovery for uranium case RO units.
            if case_study == "uranium":
                m.fs.ion_exchange.removal_fraction[0, "tds"].unfix()
                m.fs.ion_exchange.water_recovery.fix(.66)
                m.fs.ion_exchange.anion_res_capacity.unfix()
                m.fs.ion_exchange.cation_res_capacity.unfix()
            
            if scenario == "swd_well":
                m.fs.produced_water_for_dw.conc_mass_in[0, "tds"].fix(50)
            
            # run and and return baseline with 0 degrees of freedom
            m = wt.run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = wt.get_fixed_onm_reduction(m, reduction_value_list = [0.9, 0.8, 0.7, 0.6, 0.5],
                                                  skip_small=skip_small_base)

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        for key in m.fs.pfd_dict.keys():
                            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                                getattr(m.fs, key).feed.pressure.unfix()
                                getattr(m.fs, key).membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = "1p5_mgd"
                        if n == 2: s_name = "3_mgd"
                        if n == 3: s_name = "5_mgd"
                        df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1
                        
            
            if case_study == "lithium":
                ### double cap
                if scenario == "baseline": 
                    s_name = "double_cap"
                else:
                    s_name = scenario + "_double_cap"
                print("goes in")
                flow_in_x2 = m.fs.lithium_brine.flow_vol_in[0]() * 2
                m.fs.lithium_brine.flow_vol_in.fix(flow_in_x2)
                print("adjusted baseline flow -->", flow_in_x2)
                wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                df_no3 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)

            
            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                print("goes in")
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



------------------------------------------------------
------------------------------------------------------
----------- big_spring baseline -----------------
------------------------------------------------------
------------------------------------------------------
big_spring

------- Adding Unit Processes -------
onsite_storage
microfiltration
reverse_osmosis
uv_aop
treated_storage
municipal_drinking
surface_discharge
-------------------------------------

adding source
[]

Connecting unit processes...
Unfixing feed presure and area for reverse_osmosis ...

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
	Feed pressure for reverse_osmosis: 34.51 bar
	Membrane area for reverse_osmosis: 6940.29 m2
	Pure Water Flux for reverse_osmosis: 35.140865547739914 lmh
	A constant for reverse_osmosis: 4.2
	B constant for reverse_osmosis: 0.35
NON sea


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9250000000000002 LCOW --> 0.7744673073065425
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9400000000000002 LCOW --> 0.7621087864452679
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9550000000000002 LCOW --> 0.7501384913702113
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9700000000000002 LCOW --> 0.738


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.7638012500000001 LCOW --> 0.7064829807100496
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.78844 LCOW --> 0.7075919648251437
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.8130787500000001 LCOW --> 0.7086984916267832
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.8377175000000001 LCOW --> 0.7098026471857918
----------------------------------------------------------------------

D


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.07687500000000003 LCOW --> 0.7135004226844434
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.07875000000000001 LCOW --> 0.7111233065400258
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.08062500000000002 LCOW --> 0.7092578177633535
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.08250000000000002 LCOW --> 0.7079118367008866
------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.7163822592585519
------- electricity price +- 30% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.038149999999999996 LCOW --> 0.6892752953901169
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.039784999999999994 LCOW --> 0.6919859917769603
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04141999999999999 LCOW --> 0.6946966881638039
------------------------

...

------- Plant Capacity Utilization 70-100% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.7 LCOW --> 2.4748605776985797
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.715 LCOW --> 2.4229404257188945
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.73 LCOW --> 2.373153978615086
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

-------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07000000000000002 LCOW --> 2.0735353040048317
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07250000000000002 LCOW --> 2.1181827522508603
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07500000000000002 LCOW --> 2.1632342743193043
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.077500000000


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.3160425000000004 LCOW --> 1.7324024043890087
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.365320000000001 LCOW --> 1.7324024043890147
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.414597500000001 LCOW --> 1.7324024043890098
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.463875000000001 LCOW --> 1.7324024043890098
------- RESET -------
-------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 22.5 LCOW --> 1.9515502873035153
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 24.0 LCOW --> 1.8953267031861405
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 25.5 LCOW --> 1.8464056648770035
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 1.8035590023762613
----------------------------------------------------------------------

Degrees of


WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06451499999999995 LCOW --> 1.7336837451777845
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06619799999999995 LCOW --> 1.7339400133355372
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06788099999999994 LCOW --> 1.734196281493299
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06956399999999993 LCOW --> 1.7344525496510466
-----------------------

In [7]:
recovery = m.fs.reverse_osmosis.flow_vol_out[0]()/m.fs.reverse_osmosis.flow_vol_in[0]()
recovery2 = m.fs.ro_stage.flow_vol_out[0]()/m.fs.ro_stage.flow_vol_in[0]()
recovery3 = m.fs.reverse_osmosis_2.flow_vol_out[0]()/m.fs.reverse_osmosis_2.flow_vol_in[0]()

recovery

0.5279211664132168

In [8]:
recovery2

0.7288799695781575

In [9]:
recovery3

0.5049908918782277

In [8]:
m.fs.splitter1.display()

Block fs.splitter1

  Variables:
    flow_vol_outlet1 : Volumetric flowrate of water out of unit
        Size=1, Index=fs.time
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
        0.0 : 1e-09 : 0.821360036601861 : 100.0 : False : False : NonNegativeReals
    conc_mass_outlet1 : Mass concentration of species at outlet
        Size=11, Index=fs.splitter1.conc_mass_outlet1_index
        Key                : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0.0, 'boron') :  None :   0.005930116887606596 :  None : False : False :  Reals
          (0.0, 'bromide') :  None :   0.000761053031932074 :  None : False : False :  Reals
          (0.0, 'calcium') :  None :   0.004701769621574556 :  None : False : False :  Reals
         (0.0, 'chloride') :  None :     0.2199592539835969 :  None : False : False :  Reals
        (0.0, 'magnesium') :  None :    0.01467007236788649 :  None : False : False :  Reals
        (0.0, 'potassium') :  None 

In [23]:
wt.display.show_train2(model_name=m)

In [ ]:
#wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

In [18]:
m.fs.splitter1.flow_vol_in[0]()

0.057347809908287334

In [ ]:
0.006308259089911607

In [ ]:
0.051039550818375724

In [19]:
0.051039550818375724/0.057347809908287334

0.8899999999999999

In [17]:
m.fs.splitter1.display()

Block fs.splitter1

  Variables:
    flow_vol_outlet1 : Volumetric flowrate of water out of unit
        Size=1, Index=fs.time
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
        0.0 : 1e-09 : 0.006308259089911607 : 100.0 : False : False : NonNegativeReals
    conc_mass_outlet1 : Mass concentration of species at outlet
        Size=13, Index=fs.splitter1.conc_mass_outlet1_index
        Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0.0, 'alkalinity_as_caco3') :  None :    0.11705330011041193 :  None : False : False :  Reals
                        (0.0, 'calcium') :  None : 0.00035160174219506387 :  None : False : False :  Reals
                       (0.0, 'chloride') :  None :  0.0003039890062728146 :  None : False : False :  Reals
        (0.0, 'electrical_conductivity') :  None :     261.97643358044576 :  None : False : False :  Reals
                      (0.0, 'magnesium') :  None :  

In [21]:
m.fs.ozone.flow_vol_in[0]() * 15850.372483753

1169.8873365137872

In [22]:
0.051039550818375724 * 15850.372483753

808.9958918746955

In [4]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
None
